<div style="text-align: center;">
<img src="https://assets-global.website-files.com/62b9d45fb3f64842a96c9686/62d84db4aeb2f6552f3a2f78_Quantinuum%20Logo__horizontal%20blue.svg" width="200" height="200" /></div>

# Qubit Reuse Compilation

This notebook contains an overview of how to use the `pyqubit_reuse` package. This package provides a `pytket` compiler pass for applying the compilation strategy for reducing the number of qubits in a quantum circuit as detailed in [Qubit-reuse compilation with mid-circuit measurement and reset](https://journals.aps.org/prx/abstract/10.1103/PhysRevX.13.041057) by Matthew DeCross, Eli Chertkov, Megan Kohagen and Michael Foss-Feig.

`pytket` is a python module for interfacing with `TKET`, a quantum computing toolkit and optimisation compiler developed by Quantinuum, and is available through `pip`. `pytket` is open source and documentation and examples can be found in the [pytket examples](https://tket.quantinuum.com/examples/) and in the *examples* folder in the [pytket-quantinuum Github repository](https://github.com/CQCL/pytket-quantinuum).

* [Setup Nexus Project](#Setup-Nexus-Project)
* [How to Cite](#How-to-Cite)
* [Simple Example](#Simple-Example)
* [Demonstrating Compilation with Qubit Reuse for QAOA Circuits](#Demonstrating-Compilation-with-Qubit-Reuse-for-QAOA-Circuits)
* [Additional Ordering Methods](#additional-ordering-methods)

## How to Cite

If you wish to cite the `pyqubit_reuse` package in any academic publications, we recommend citing our paper [Qubit-reuse compilation with mid-circuit measurement and reset](https://journals.aps.org/prx/abstract/10.1103/PhysRevX.13.041057).

## Setup Nexus Project

Nexus enables access to H-Series. An existing project is activated, **Qubit-Reuse-Demonstration**.

In [ ]:
import qnexus

In [ ]:
project = qnexus.projects.get_or_create(name="Qubit-Reuse-Demonstration")

In [ ]:
job_name_suffix = qnexus.jobs.datetime.now().strftime("%Y_%m_%d-%H-%M-%S")

In [ ]:
qnexus.context.set_active_project(project)

The `QuantinuumConfig` is instantiated below to backend_config the `H1-Emulator`, an emulator instance of *System Model H1*, that is hosted on nexus.

In [ ]:
config = qnexus.QuantinuumConfig(device_name="H1-Emulator")

## Simple Example

Most current techniques for circuit optimisation focus on reducing the number of gates in a circuit, often aiming to reduce the number of multi-qubit gates as they are known to be more error-prone. The prevailing logic is that a shorter circuit accumulates less noise and so provides better results. The compilation technique available in this repository instead focuses on reducing the number of qubits, or width, of a circuit. This can help turn a circuit that at first seems infeasible on small near-term devices into one that can be executed. A full explanation of the techniques available can be read in the paper [Qubit-reuse compilation with mid-circuit measurement and reset](https://journals.aps.org/prx/abstract/10.1103/PhysRevX.13.041057).

Let's first consider how such a technique is possible via a basic example. An existing circuit is constructed and uploaded to nexus.

In [ ]:
from pytket import Circuit
from pytket.circuit.display import render_circuit_jupyter

circuit = (
    Circuit(3, 3)
    .H(0)
    .CX(0, 1)
    .CX(1, 2)
    .Measure(0, 0)
    .Measure(1, 1)
    .Measure(2, 2)
)

render_circuit_jupyter(circuit)

In [ ]:
ref_circuit = qnexus.circuits.upload(circuit, name="simple-circuit")

### Compilation with Nexus

The circuit above is compiled remotely with nexus. The `optimisation_level` is set to `0` and the compiled circuit only satisfies the gate-set predicate for the nexus-hosted H1 emulator.

In [ ]:
ref_compile_job = qnexus.start_compile_job(
    circuits=[ref_circuit],
    name=f"compile-simple-circuit-{job_name_suffix}",
    backend_config=config,
    optimisation_level=0
)

In [ ]:
qnexus.jobs.wait_for(ref_compile_job)

In [ ]:
job_result = qnexus.jobs.results(ref_compile_job)[0]

In [ ]:
compilation_result_ref = job_result.get_output()

In [ ]:
compiled_circuit = compilation_result_ref.download_circuit()

In [ ]:
from pytket.circuit.display import render_circuit_jupyter

render_circuit_jupyter(compiled_circuit)

### Execution

The compiled circuit is submitted to nexus-hosted H1-Emulator with 100 shots.

In [ ]:
execution_ref = qnexus.start_execute_job(
    circuits=[compilation_result_ref], # <- Don't have to download, can just use the reference.
    name=f"execution-simple-circuit-{job_name_suffix}",
    backend_config=config,
    n_shots=[100],
)

In [ ]:
qnexus.jobs.wait_for(execution_ref)

In [ ]:
result = qnexus.jobs.results(execution_ref)[0].download_result()

In [ ]:
result.get_distribution()

However, looking at the Circuit we can see that the `CX` gate targeting `q[2]` is executed after all the operations on `q[0]`. As an alternative Circuit, we could measure `q[0]`, reset it to the `0` state and then replace the `CX` gate originally between `q[1]` and `q[2]` with between `q[1]` and `q[0]`. A circuit is constructed with these modified instructions and uploaded to nexus.

In [ ]:
from pytket import OpType

circuit = (
    Circuit(2, 3)
    .H(0)
    .CX(0, 1)
    .Measure(0, 0)
    .add_gate(OpType.Reset, [0])
    .CX(1, 0)
    .Measure(1, 1)
    .Measure(0, 2)
)
render_circuit_jupyter(circuit)

In [ ]:
ref_circuit_2 = qnexus.circuits.upload(circuit, name="simple-circuit-2")

In [ ]:
compile_job_ref = qnexus.start_compile_job(
    circuits=[ref_circuit_2], 
    name=f"compile-job-simple-circuit-2-{job_name_suffix}", 
    backend_config=config, 
    optimisation_level=0
)

In [ ]:
qnexus.jobs.wait_for(compile_job_ref)
compile_result = qnexus.jobs.results(compile_job_ref)[0]
compile_ref_circuit = compile_result.get_output()

In [ ]:
execute_job_ref = qnexus.start_execute_job(
    circuits=[compile_ref_circuit],
    name=f"execute-simple-circuit-2-{job_name_suffix}",
    backend_config=qnexus.QuantinuumConfig(device_name="H1-Emulator"),
    n_shots=[100]
)
qnexus.jobs.wait_for(execute_job_ref)

In [ ]:
result = qnexus.jobs.results(execute_job_ref)[0].download_result()

In [ ]:
result.get_distribution()

In this case, by reusing `q[0]` we've completed the same circuit using one fewer qubit.

This repository provides a `pytket` compiler pass for automatically applying qubit reuse schemes to `pytket` Circuit objects. If you are unfamiliar with compilation in `pytket` the [compilation example notebook](https://github.com/CQCL/pytket/blob/main/examples/compilation_example.ipynb) in the `pytket` repository covers the basics.

Lets construct a `QubitReuse` `CompilerPass` object and apply it to our original 3-qubit circuit.

One of the arguments for constructing the `CompilerPass` is a function for ordering. The reuse compilation works by finding causal cones of qubit outputs (See Fig. 2 in [Qubit-reuse compilation with mid-circuit measurement and reset](https://journals.aps.org/prx/abstract/10.1103/PhysRevX.13.041057)) and implementing all of the gates in a given causal cone before proceeding to other causal cones. The order in which causal cones are chosen for implementation determines how many qubits are required in the output circuit, since causal cones that share many qubits generally require adding fewer new live qubits to a circuit to implement. An optimal ordering of causal cones produces a circuit with a minimum number of qubits.

The `OrderingMethod` class provides four options for finding this ordering. For now we will look at the `BruteForceOrder` option.

In [ ]:
from pytket.circuit import Circuit
from pytket.circuit.display import render_circuit_jupyter
from pytket.predicates import CompilationUnit

from pyqubit_reuse import QubitReuse, OrderingMethod

circuit = (
    Circuit(3, 3)
    .H(0).CX(0, 1)
    .CX(1, 2)
    .Measure(0, 0)
    .Measure(1, 1)
    .Measure(2, 2)
)

render_circuit_jupyter(circuit)

qubit_reuse_pass = QubitReuse(OrderingMethod.BruteForceOrder())
compilation_unit = CompilationUnit(circuit)
qubit_reuse_pass.apply(compilation_unit)
render_circuit_jupyter(compilation_unit.circuit)

As with the case we solved ourselves above, the `QubitReuse` pass is able to reduce the circuit from three to two qubits.

Now let's consider a real algorithm.

## Demonstrating Compilation with Qubit Reuse for QAOA Circuits

Combinatorial optimization problems in quantum computing can be considered as the problem of finding the ground state and its energy for a diagonal Hamiltonian on $n$ qubits:

$H = \sum\limits_{x\in\{0,1\}^{n}}C(x)|x\rangle\langle x|$ 

The paper [A Quantum Approximate Optimisation Algorithim](https://arxiv.org/abs/1411.4028) by Edward Farhi, Jeffrey Goldstone and Sam Gutmann details an algorithm for doing this.

A commonly tackled combinatorial optimization problem is the Max Cut problem: given some graph with $N$ nodes and $E$ edges, split the nodes into two subsets such that there is a maximum number of edges between both subsets. This is equivalent to finding the ground state and its energy for the following Hamiltonian: 

$H = -\sum\limits_{(j,k)\in E} 0.5(1 - Z_jZ_k)$ 

Since the first term in parentheses shifts the energy by an overall constant, it can be ignored.

Let's create a graph and consider an example problem using `networkx`.

**Note:** To run this example, run `pip install networkx` in your python environment.

In [ ]:
import networkx as nx

max_cut_graph_edges = [(0, 1), (1, 2), (1, 3), (3, 4), (4, 5), (4, 6)]
max_cut_graph = nx.Graph()
max_cut_graph.add_edges_from(max_cut_graph_edges)
nx.draw(max_cut_graph, labels={node: node for node in max_cut_graph.nodes()})

For this problem, having nodes `1` and `4` in the same subset (i.e. labelled differently to the remaining nodes) gives 6 edges between subsets, the maximum result.

[A Quantum Approximate Optimisation Algorithim](https://arxiv.org/abs/1411.4028) uses a variational algorithim with a parameterised circuit construction to find the maximum eigenvalue and corresponding eigenstates of the encoded Hamiltonian. From here on we will not look at how to solve the problem, but instead take the circuit construction proposed and show how qubit reuse can reduce the number of qubits in the circuit. 

To do so, we will define a function to convert the edges of a graph to a `pytket` QAOA `Circuit`. Technically, qubit reuse could depend on the order in which we insert gates corresponding to edges, but we will not worry about that for this demonstration.

In [ ]:
from typing import List, Tuple

def gen_qaoa_max_cut_circuit(edges: List[Tuple[int, int]],
                             n_nodes: int,
                             mixer_angles: List[float],
                             cost_angles: List[float],) -> Circuit:
    """ Generate QAOA MaxCut circuit. """
    assert len(mixer_angles) == len(cost_angles)

    # initial state
    qaoa_circuit = Circuit(n_nodes)
    for qubit in range(n_nodes):
        qaoa_circuit.H(qubit)

    # add cost and mixer terms to state
    for cost, mixer in zip(cost_angles, mixer_angles):

        for edge in edges:
            qaoa_circuit.ZZPhase(cost, edge[0], edge[1])

        for i in range(n_nodes):
            qaoa_circuit.Rx(mixer,i)

    qaoa_circuit.measure_all()
    return qaoa_circuit

The `mixer_angles` and `cost_angles` inputs are the parameterised values used to explore the solution space and find the maximum eigenvalue and corresponding eigenstates. For this example, which is only considering qubit reuse, we will set them arbitrarily to values that won't be optimised away.

In [ ]:
qaoa_circuit = gen_qaoa_max_cut_circuit(max_cut_graph_edges, 7, [0.3], [0.3])
render_circuit_jupyter(qaoa_circuit)

We've constructed a 7-qubit circuit: an intuitive explanation of the circuit constructed may be that each edge in the graph corresponds to a ZZ term in the Hamiltonian, and each ZZ term in the Hamiltonian corresponds to a ZZPhase gate in the circuit.

Can qubit reuse reduce this? 

Here the `BruteForceOrder` `OrderingMethod` is used. This will take a long time to run since it will calculate and score every possible ordering for merging causal cones, with the number of orders scaling exponentially.

In [ ]:
compilation_unit = CompilationUnit(qaoa_circuit)
qubit_reuse_pass.apply(compilation_unit)
render_circuit_jupyter(compilation_unit.circuit)

In [ ]:
print("Reused Circuit has", compilation_unit.circuit.n_qubits, "qubits.")

By applying the qubit reuse pass we're able to reduce the number of qubits in the Circuit from 7 to 2. 

Lets consider a larger, random problem.

In [ ]:
random_9_node_graph = nx.random_regular_graph(4, 9)
nx.draw(
    random_9_node_graph, labels={node: node for node in random_9_node_graph.nodes()}
)

In [ ]:
random_9_node_graph_edges = list(random_9_node_graph.edges())
qaoa_circuit_9 = gen_qaoa_max_cut_circuit(random_9_node_graph_edges, 9, [0.3], [0.3])
render_circuit_jupyter(qaoa_circuit_9)

In [ ]:
compilation_unit = CompilationUnit(qaoa_circuit_9)
qubit_reuse_pass.apply(compilation_unit)
render_circuit_jupyter(compilation_unit.circuit)

In [ ]:
print("Reused Circuit has", compilation_unit.circuit.n_qubits, "qubits.")

Given this, the qubit reuse package comes with several other `OrderingMethod` functions for finding solutions for larger problems. Lets see how each of them perform for a much larger problem.

## Additional Ordering Methods

Note that all the functions provided below correspond to the techniques outlined in [Qubit-reuse compilation with mid-circuit measurement and reset](https://journals.aps.org/prx/abstract/10.1103/PhysRevX.13.041057).

In [ ]:
random_100_node_graph = nx.random_regular_graph(5, 100)
nx.draw(
    random_100_node_graph, labels={node: node for node in random_100_node_graph.nodes()}
)

In [ ]:
random_100_node_graph_edges = list(random_100_node_graph.edges())
qaoa_circuit_100 = gen_qaoa_max_cut_circuit(
    random_100_node_graph_edges, 100, [0.3], [0.3]
)

Using `OrderingMethod.LocalGreedyOrder()`.

In [ ]:
compilation_unit = CompilationUnit(qaoa_circuit_100)
qubit_reuse_pass_local_greedy = QubitReuse(OrderingMethod.LocalGreedyOrder())
qubit_reuse_pass_local_greedy.apply(compilation_unit)
print("Reused Circuit has", compilation_unit.circuit.n_qubits, "qubits.")

Using `OrderingMethod.LocalGreedyFirstNodeSearchOrder()`.

In [ ]:
compilation_unit = CompilationUnit(qaoa_circuit_100)
qubit_reuse_pass_local_greedy_first_node_search_order = QubitReuse(
    OrderingMethod.LocalGreedyFirstNodeSearchOrder()
)
qubit_reuse_pass_local_greedy_first_node_search_order.apply(compilation_unit)
print("Reused Circuit has", compilation_unit.circuit.n_qubits, "qubits.")

The default settings for qubit reuse will use `OrderingMethod.ConstrainedOptOrder()` for qubit numbers between 10 and 25. The `ConstrainedOptOrder` function corresponds to the CP-SAT method outlined in the paper, which finds the optimal reduced number of qubits. The complexity of CP-SAT is highly dependent on circuit structure, so it may or may not take a long time for larger circuits.

In [ ]:
random_20_node_graph = nx.random_regular_graph(4, 20)
nx.draw(
    random_20_node_graph, labels={node: node for node in random_20_node_graph.nodes()}
)

In [ ]:
random_20_node_graph_edges = list(random_20_node_graph.edges())
qaoa_circuit_20 = gen_qaoa_max_cut_circuit(random_20_node_graph_edges, 20, [0.3], [0.3])
render_circuit_jupyter(qaoa_circuit_20)

In [ ]:
compilation_unit = CompilationUnit(qaoa_circuit_20)
qubit_reuse_pass_constrained_opt_order = QubitReuse(
    OrderingMethod.ConstrainedOptOrder()
)
qubit_reuse_pass_constrained_opt_order.apply(compilation_unit)
render_circuit_jupyter(compilation_unit.circuit)
print("Reused Circuit has", compilation_unit.circuit.n_qubits, "qubits.")

`OrderingMethod.DefaultOrder()` switches between the four ordering methods shown depending on the number of qubits.

In [ ]:
compilation_unit = CompilationUnit(qaoa_circuit_20)
qubit_reuse_pass_default_order = QubitReuse(OrderingMethod.DefaultOrder())
qubit_reuse_pass_default_order.apply(compilation_unit)
render_circuit_jupyter(compilation_unit.circuit)
print("Reused Circuit has", compilation_unit.circuit.n_qubits, "qubits.")

Finally, there is `OrderingMethod.CustomOrder()` which uses an order specified at construction.

In [ ]:
compilation_unit = CompilationUnit(qaoa_circuit_20)
qubit_reuse_pass_default_order = QubitReuse(OrderingMethod.CustomOrder(range(0, 20)))
qubit_reuse_pass_default_order.apply(compilation_unit)
render_circuit_jupyter(compilation_unit.circuit)
print("Reused Circuit has", compilation_unit.circuit.n_qubits, "qubits.")

The `QubitReuse` pass has two further parameters that can be used. 

The first is `DualStrat`. Given that some of the available reodering methods are heuristics, sometimes a better solution can be found by running the algorithm on the dual circuit as described in [Qubit-reuse compilation with mid-circuit measurement and reset](https://journals.aps.org/prx/abstract/10.1103/PhysRevX.13.041057). In this case, we can run the qubit reuse scheme on the reverse (dual) circuit and then reverse it again to get a logically equivalent circuit. 

`DualStrat` has three options: `Single`, `Dual` and `Auto`. `Single` runs the qubit reuse algorithm on the given circuit only. `Dual` runs the qubit reuse algorithm on the reversed circuit and `Auto` runs on both the given and reversed circuit and returns the circuit with fewer qubits. If both circuits have the same number of qubits it returns the circuit with better depth, defaulting to the `None` strategy if both are deemed equivalent.  By default, and for best performance, `Auto` is used. 

We will look at how to run it using the previous QAOA examples.

In [ ]:
from pyqubit_reuse import DualStrat

compilation_unit = CompilationUnit(qaoa_circuit_20)
qubit_reuse_pass_default_order = QubitReuse(
    OrderingMethod.DefaultOrder(), dual_strat=DualStrat.Dual
)
qubit_reuse_pass_default_order.apply(compilation_unit)
render_circuit_jupyter(compilation_unit.circuit)
print("Reused Circuit has", compilation_unit.circuit.n_qubits, "qubits.")

In [ ]:
print("Reused Circuit has", compilation_unit.circuit.n_gates, "gates.")

We can also set a minimum number of qubits as a target for the qubit reuse algorithm, which guarantees that the returned circuit whose number of qubits is equals to or less than the setting qubit number.

We can take the QAOA problem we have just run and specify a minimum number of qubits larger than the 9 it returned.

In [ ]:
compilation_unit = CompilationUnit(qaoa_circuit_20)
qubit_reuse_pass_default_order = QubitReuse(
    OrderingMethod.DefaultOrder(), dual_strat=DualStrat.Dual, min_qubits=11
)
qubit_reuse_pass_default_order.apply(compilation_unit)
render_circuit_jupyter(compilation_unit.circuit)
print("Reused Circuit has", compilation_unit.circuit.n_qubits, "qubits.")

In [ ]:
print("Reused Circuit has", compilation_unit.circuit.n_gates, "gates.")

We can see that by specifying a minimum of 11 qubits, the returned circuit is now larger. 

Finally, in some algorithmic scenarios, such as computing few-body correlation functions, we may be able to reason that the outcome results of certain qubits are not necessary to the full algorithm being run. For this scenario we provide a method in `pyqubit_reuse` called `correlation_subcircuit` which given a list of `Qubit` and `Bit` that are known to be required, returns a new `Circuit` only containing operations in the causal cones of the `Qubit` and `Bit` given. 

In [ ]:
from pyqubit_reuse import correlation_subcircuit
from pytket import Bit

correlated_qaoa_20 = correlation_subcircuit(
    qaoa_circuit_20, [Bit(i) for i in range(0, 5)]
)

render_circuit_jupyter(correlated_qaoa_20)
print("Correlated subcircuit has", correlated_qaoa_20.n_qubits, "qubits.")

We can then apply a `QubitReuse` pass to this.

In [ ]:
compilation_unit = CompilationUnit(correlated_qaoa_20)
qubit_reuse_pass_default_order = QubitReuse(OrderingMethod.DefaultOrder())
qubit_reuse_pass_default_order.apply(compilation_unit)
render_circuit_jupyter(compilation_unit.circuit)
print("Reused Circuit has", compilation_unit.circuit.n_qubits, "qubits.")

As described in [Qubit-reuse compilation with mid-circuit measurement and reset](https://journals.aps.org/prx/abstract/10.1103/PhysRevX.13.041057), compilation of tensor network circuits can be improved with qubit-reuse. For instance, for a quantum circuit implementation of the (depth-D, binary, open boundary conditions) Multiscale Entanglement Renormalization Ansatz (MERA, see [A class of quantum many-body states that can be efficiently simulated](https://arxiv.org/pdf/quant-ph/0610099) in detail) defined on $2^D$ qubits, the full output can be measured with only $2D - 1$ qubits.

In [ ]:
def isometry(circ: Circuit, l_Q: List[int]) -> None:
    """ Adds isometry circuit by modifying circ. """
    for i in range(0, len(l_Q), 2): 
        circ.U3(1/2, 1/2, 1/2, l_Q[i])
        circ.U3(1/2, 1/2, 1/2, l_Q[i+1])
        circ.CX(l_Q[i+1], l_Q[i])
        circ.Rz(1/2, l_Q[i])
        circ.Ry(1/2, l_Q[i+1])
        circ.CX(l_Q[i], l_Q[i+1])
        circ.Ry(1/2, l_Q[i+1])
        circ.CX(l_Q[i+1], l_Q[i])
        circ.U3(1/2, 1/2, 1/2, l_Q[i])
        circ.U3(1/2, 1/2, 1/2, l_Q[i+1])
        

def entangler(circ: Circuit, l_Q: List[int]) -> None:
    """ Adds entangler circuit by modifying circ. """
    for i in range(0, len(l_Q)-2, 2): 
        circ.U3(1/2, 1/2, 1/2, l_Q[i+1])
        circ.U3(1/2, 1/2, 1/2, l_Q[i+2])
        circ.CX(l_Q[i+2], l_Q[i+1])
        circ.Rz(1/2, l_Q[i+1])
        circ.Ry(1/2, l_Q[i+2])
        circ.CX(l_Q[i+1], l_Q[i+2])
        circ.Ry(1/2, l_Q[i+2])
        circ.CX(l_Q[i+2], l_Q[i+1])
        circ.U3(1/2, 1/2, 1/2, l_Q[i+1])
        circ.U3(1/2, 1/2, 1/2, l_Q[i+2])

def qMERA_circuit(depth: int) -> Circuit:
    """ Given a depth, returns a Multiscale Entangelemnt Renormalization Ansatz circuit."""
    n_qubits = 2**depth
    circ = Circuit(n_qubits, n_qubits)
    for j in range(depth):
        list_qb: List[int] = [i_nq for i_nq in range(0, n_qubits, n_qubits//(2**(j+1)))]    
        isometry(circ, list_qb)
        entangler(circ, list_qb)
     
    circ.measure_all()
    return circ


In [ ]:
compilation_unit = CompilationUnit(qMERA_circuit(7))
print("Original circuit has", compilation_unit.circuit.n_qubits, "qubits.")
QubitReuse(OrderingMethod.LocalGreedyFirstNodeSearchOrder()).apply(compilation_unit)
print("Reused Circuit has", compilation_unit.circuit.n_qubits, "qubits.")

In the above, we took a MERA circuit originally defined on $2^7 = 128$ qubits and compressed it down to only require $2*7 - 1 = 13$ qubits to measure the full output, which agrees with the analytic result for optimal compression. Notice that the compilation in the above cell was performed with the heuristic `OrderingMethod.LocalGreedyFirstNodeSearchOrder()` but still returned the optimal result, demonstrating that the heuristic is able to capture optimal results on problems too large for brute force solvers.

In many practical applications like MERA, one is only interested in measuring local correlation functions like $\langle X_0 X_{64}\rangle$, requiring measurements on only a subset of the output qubits. For instance, suppose one is only interested in measuring such a correlation function involving qubits 0 and 64 in the MERA defined above. In that case, we can first define the restricted `correlation_subcircuit` corresponding to those circuits, before applying the qubit reuse pass to the restricted subcircuit.

In [ ]:
correlated_qMera = correlation_subcircuit(
    qMERA_circuit(7), [Bit(0), Bit(64)]
)
compilation_unit = CompilationUnit(correlated_qMera)
QubitReuse(OrderingMethod.LocalGreedyFirstNodeSearchOrder()).apply(compilation_unit)
print("Reused Circuit has", compilation_unit.circuit.n_qubits, "qubits.")

Measuring the output of this restricted subcircuit requires only 4 qubits on hardware, less than the optimal compression of 13 that the entire output would require.

<div align="center"> &copy; 2024 by Quantinuum. All Rights Reserved. </div>